<a href="https://colab.research.google.com/github/BotenAqua/WarframeInventoryHelper/blob/master/WarframeInventoryHelper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import requests as rq
import pandas as pd

In [93]:
def wikiScrapper():
  page = rq.get("https://warframe.fandom.com/wiki/Void_Relic/ByRewards/SimpleTable")
  return page.text

def extractData(page):
  page = page[page.find("<table"):page.find("</table")]
  page = page[page.find('">')+2:]

  #colnames

  h_end = page.find("</tr>")
  headersT = str(page[:h_end])
  headers = ["url"]
  for colname in range(headersT.count("col")):
    do = headersT.find("</th>")
    headers.append(headersT[headersT.find('">')+2:do-1])
    headersT = headersT[do+4:]
  del headersT

  #data

  page = page[h_end+5:]
  drop = []
  #print(page)
  ograniczenia = [['f="', '" t'],['">', '</a></td>'], ["<td>", "</td>"], ["<td>", "</td>"], ['param="', '"><a'], ["</span></td><td>", "</td></tr>"]]
  for _ in range(page.count("<td><a href=")):
    item = []
    for el in ograniczenia:
      do = page.find(el[1])
      #print(page.find(el[0])+len(el[0]),do)
      item.append(page[page.find(el[0])+len(el[0]):do])
      page = page[do+len(el[1]):]
    drop.append(item)
    page = page[page.find("</tr>")+5:]
  for el in drop:
    f = el[5].find("</td><td>")
    el.append(el[5][f+9:-1])
    el[5] = el[5][:f]

  #dataframe

  data = pd.DataFrame(drop)
  data.columns = headers
  return data
dane = wikiScrapper()
dane = extractData(dane)
print(dane)

                         url              Item  ...    Rarity Vaulted?
0        /wiki/Akbolto_Prime     Akbolto Prime  ...  Uncommon       No
1        /wiki/Akbolto_Prime     Akbolto Prime  ...  Uncommon      Yes
2        /wiki/Akbolto_Prime     Akbolto Prime  ...    Common       No
3        /wiki/Akbolto_Prime     Akbolto Prime  ...    Common      Yes
4        /wiki/Akbolto_Prime     Akbolto Prime  ...  Uncommon       No
5        /wiki/Akbolto_Prime     Akbolto Prime  ...  Uncommon      Yes
6        /wiki/Akbolto_Prime     Akbolto Prime  ...  Uncommon      Yes
7        /wiki/Akbolto_Prime     Akbolto Prime  ...      Rare       No
8        /wiki/Akbolto_Prime     Akbolto Prime  ...      Rare      Yes
9       /wiki/Akbronco_Prime    Akbronco Prime  ...    Common       No
10      /wiki/Akbronco_Prime    Akbronco Prime  ...    Common      Yes
11      /wiki/Akbronco_Prime    Akbronco Prime  ...    Common      Yes
12      /wiki/Akbronco_Prime    Akbronco Prime  ...    Common      Yes
13    